In [ ]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [ ]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['image_20.jpg',
 'image_60.jpg',
 'image_180.jpg',
 'models',
 'FaceImage',
 'ok_data_face_features.pickle',
 'project']

In [ ]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/ok_data_face_features.pickle", mode="rb"))

In [ ]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 2.64238715e-02  2.36892134e-01  3.72064002e-02 ... -6.46553487e-02
    1.20319806e-01  4.71187644e-02]]

 [[ 4.03175279e-02  1.54421568e-01 -6.44083992e-02 ... -2.61096098e-02
    4.62537520e-02  6.70112148e-02]]

 [[ 1.66380070e-02  2.40359947e-01 -2.34857565e-04 ... -4.78317812e-02
    1.08082436e-01  6.52680472e-02]]

 ...

 [[ 2.10880283e-02  1.73503324e-01  8.18642527e-02 ... -3.02533503e-03
    1.54735163e-01  8.87988508e-02]]

 [[-1.00170225e-02  1.87323347e-01  3.37029286e-02 ...  1.23106493e-02
    1.97563559e-01  8.30874294e-02]]

 [[-3.36680934e-02  1.74611300e-01  1.92929655e-02 ...  3.55110094e-02
    1.83929533e-01  9.85852331e-02]]]
['1721031229_NinhVanDung' '1721031229_NinhVanDung'
 '1721031229_NinhVanDung' ... '1721030936-phamtrantrunghieu'
 '1721030936-phamtrantrunghieu' '1721030936-phamtrantrunghieu']


In [ ]:
x.shape

(4686, 1, 128)

In [ ]:
y.shape

(4686,)

In [ ]:
x = x.reshape(-1, 128)

In [ ]:
x.shape

(4686, 128)

In [ ]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3748, 128), (938, 128), (3748,), (938,))

In [ ]:
# 2. Train machine learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [ ]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [ ]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [ ]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.71
Accurency Test = 0.70
F1 Score Train = 0.51
F1 Score Test = 0.50


In [ ]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [ ]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.85
F1 Score Train = 0.83
F1 Score Test = 0.74


In [ ]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [ ]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.66


In [ ]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])


In [ ]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [ ]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.97
Accurency Test = 0.84
F1 Score Train = 0.97
F1 Score Test = 0.76


In [ ]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [ ]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  10.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   8.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   8.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   6.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.3s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [29]:
model_best_estimator = model_grid.best_estimator_

In [30]:
model_grid.best_score_

0.8356419535628503

In [31]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))